# LangGraph Chatbot with Ollama

This notebook demonstrates how to create a conversational AI agent using LangGraph and Ollama.

## Prerequisites
- Install required packages: `pip install langgraph langchain-community langchain-core`
- Install and run Ollama locally
- Pull the Gemma model: `ollama pull gemma:2b`

---

## 1. Import Required Libraries

First, we import all the necessary components for building our LangGraph agent.

In [ ]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_community.chat_models import ChatOllama
from typing import TypedDict, Annotated, Sequence
from operator import add
import json
import time

## 2. Define Agent State

The `AgentState` defines the structure of data that flows through our graph. It contains a sequence of messages that can be combined using the `add` operator.

In [ ]:
class AgentState(TypedDict):
    """State structure for the conversational agent."""
    messages: Annotated[Sequence[HumanMessage | AIMessage], add]

print("✅ AgentState defined successfully")

## 3. Setup Language Model and System Prompt

We configure the Ollama model and define a system prompt that guides the AI's behavior.

In [ ]:
# Initialize the language model
llm_model = ChatOllama(model="gemma:2b", temperature=0.7)

# Define system prompt
sys_prompt = SystemMessage(
    content="You are Agent 007, a helpful and sophisticated AI assistant. "
            "Answer questions clearly, professionally, and with a touch of wit when appropriate."
)

print("✅ Language model and system prompt configured")
print(f"📝 Model: {llm_model.model}")
print(f"🌡️ Temperature: {llm_model.temperature}")

## 4. Define the LLM Node Function

This function processes the conversation state through the language model and returns the AI's response.

In [ ]:
def llm_node(state: AgentState) -> AgentState:
    """
    Process the current state through the LLM and return the response.
    
    Args:
        state: Current agent state containing message history
        
    Returns:
        Updated state with the LLM's response
    """
    # Combine system prompt with message history
    messages = [sys_prompt] + list(state["messages"])
    
    print(f"🤔 Processing {len(state['messages'])} messages...")
    
    # Get response from the model
    response = llm_model.invoke(messages)
    
    # Return the response in the correct format
    return {"messages": [response]}

print("✅ LLM node function defined")

## 5. Create and Compile the Graph

Now we build the state graph that defines our agent's workflow.

In [ ]:
def create_agent():
    """
    Create and compile the LangGraph agent.
    
    Returns:
        Compiled agent ready for use
    """
    # Initialize the state graph
    graph = StateGraph(AgentState)
    
    # Add the LLM node
    graph.add_node("llm_model", llm_node)
    
    # Set entry point
    graph.set_entry_point("llm_model")
    
    # Add finish point
    graph.add_edge("llm_model", END)
    
    # Compile the graph
    return graph.compile()

# Create the bot instance
bot = create_agent()
print("✅ Agent created and compiled successfully!")
print("🤖 Agent 007 is ready for action!")

## 6. Test with a Single Question

Let's test our agent with a simple question to make sure everything is working.

In [ ]:
def ask_bot(question: str, conversation_history=None):
    """
    Ask the bot a single question.
    
    Args:
        question: The question to ask
        conversation_history: Optional previous conversation history
        
    Returns:
        The bot's response as a string
    """
    if conversation_history is None:
        conversation_history = []
    
    # Add the new question
    user_msg = HumanMessage(content=question)
    messages = conversation_history + [user_msg]
    
    # Get response
    result = bot.invoke({"messages": messages})
    return result["messages"][-1].content

# Test the bot
test_question = "Hello! Who are you and what can you do?"
print(f"👤 User: {test_question}")
print("\n" + "="*50)

response = ask_bot(test_question)
print(f"🤖 Agent 007: {response}")

## 7. Demo Conversation

Let's run a demonstration conversation to see how the agent maintains context.

In [ ]:
def demo_conversation():
    """
    Run a demo conversation to test the bot with context.
    """
    print("🧪 Demo Conversation with Context")
    print("=" * 40)
    
    questions = [
        "My name is Sarah and I'm learning about AI.",
        "What's the difference between machine learning and deep learning?",
        "Can you remember my name from earlier?",
        "Give me a simple example of how neural networks work."
    ]
    
    history = []
    
    for i, question in enumerate(questions, 1):
        print(f"\n🔸 Turn {i}:")
        print(f"👤 Sarah: {question}")
        
        # Add user message to history
        user_msg = HumanMessage(content=question)
        history.append(user_msg)
        
        # Get bot response
        result = bot.invoke({"messages": history})
        ai_response = result["messages"][-1]
        
        print(f"🤖 Agent 007: {ai_response.content}")
        print("-" * 40)
        
        # Add AI response to history
        history.append(ai_response)
        
        # Small delay for readability
        time.sleep(1)

# Run the demo
demo_conversation()

## 8. Interactive Chat Function

This function provides a full interactive chat experience. **Note:** This works best when run in a local Jupyter environment.

In [ ]:
def run_interactive_chat():
    """
    Run the interactive chatbot loop.
    Note: This function works best in local Jupyter notebooks.
    """
    print("🤖 Agent 007 Interactive Chat")
    print("Type your messages below. Type 'exit', 'bye', or 'quit' to end the conversation.")
    print("=" * 60)
    
    history = []
    turn_count = 0

    while True:
        try:
            # Get user input
            user_input = input(f"\n[Turn {turn_count + 1}] 👤 You: ").strip()
            
            # Check for exit conditions
            if user_input.lower() in ["exit", "bye", "quit", "stop"]:
                print("\n🤖 Agent 007: Until we meet again. Stay safe out there! 🕴️")
                break
            
            if not user_input:
                print("Please enter a message or type 'exit' to quit.")
                continue

            # Create user message and add to history
            user_msg = HumanMessage(content=user_input)
            history.append(user_msg)

            # Get bot response
            print("🤔 Agent 007 is thinking...")
            result = bot.invoke({"messages": history})
            ai_msg = result["messages"][-1]
            
            # Display response
            print(f"\n🤖 Agent 007: {ai_msg.content}")

            # Add AI response to history
            history.append(ai_msg)
            turn_count += 1
            
        except KeyboardInterrupt:
            print("\n\n🤖 Agent 007: Mission interrupted. Farewell! 🕴️")
            break
        except Exception as e:
            print(f"❌ Error: {e}")
            print("Please try again or type 'exit' to quit.")

print("✅ Interactive chat function ready!")
print("💡 To start chatting, run: run_interactive_chat()")

## 9. Utility Functions

Additional helper functions for analyzing conversations and managing the bot.

In [ ]:
def analyze_conversation(history):
    """
    Analyze a conversation history.
    
    Args:
        history: List of messages from a conversation
    """
    if not history:
        print("No conversation history to analyze.")
        return
    
    human_messages = [msg for msg in history if isinstance(msg, HumanMessage)]
    ai_messages = [msg for msg in history if isinstance(msg, AIMessage)]
    
    print("📊 Conversation Analysis")
    print("=" * 25)
    print(f"Total messages: {len(history)}")
    print(f"Human messages: {len(human_messages)}")
    print(f"AI messages: {len(ai_messages)}")
    
    if human_messages:
        avg_human_length = sum(len(msg.content) for msg in human_messages) / len(human_messages)
        print(f"Average human message length: {avg_human_length:.1f} characters")
    
    if ai_messages:
        avg_ai_length = sum(len(msg.content) for msg in ai_messages) / len(ai_messages)
        print(f"Average AI message length: {avg_ai_length:.1f} characters")

def save_conversation(history, filename="conversation.json"):
    """
    Save conversation history to a JSON file.
    
    Args:
        history: List of messages
        filename: Output filename
    """
    conversation_data = []
    
    for msg in history:
        msg_data = {
            "type": "human" if isinstance(msg, HumanMessage) else "ai",
            "content": msg.content,
            "timestamp": time.time()
        }
        conversation_data.append(msg_data)
    
    with open(filename, 'w') as f:
        json.dump(conversation_data, f, indent=2)
    
    print(f"💾 Conversation saved to {filename}")

def quick_chat(questions_list):
    """
    Quick chat with a predefined list of questions.
    
    Args:
        questions_list: List of questions to ask
    """
    print("⚡ Quick Chat Session")
    print("=" * 20)
    
    history = []
    
    for i, question in enumerate(questions_list, 1):
        print(f"\n🔸 Question {i}: {question}")
        
        user_msg = HumanMessage(content=question)
        history.append(user_msg)
        
        result = bot.invoke({"messages": history})
        ai_response = result["messages"][-1]
        
        print(f"🤖 Response: {ai_response.content}")
        print("-" * 40)
        
        history.append(ai_response)
    
    return history

print("✅ Utility functions loaded!")

## 10. Example Usage

Here are some examples of how to use the different functions we've created.

In [ ]:
# Example 1: Quick single question
print("📝 Example 1: Single Question")
response = ask_bot("What's the weather like in space?")
print(f"Answer: {response}\n")

# Example 2: Quick chat with predefined questions
print("📝 Example 2: Quick Chat")
sample_questions = [
    "Hello, I'm interested in learning about Python programming.",
    "What are the most important Python libraries for data science?",
    "Can you give me a simple Python code example?"
]

conversation_history = quick_chat(sample_questions)

# Example 3: Analyze the conversation
print("\n📝 Example 3: Conversation Analysis")
analyze_conversation(conversation_history)

## 11. Available Commands Summary

Here's a summary of all the functions you can use:

In [ ]:
print("🎯 Available Commands:")
print("=" * 30)
print("1. ask_bot('your question')           - Ask a single question")
print("2. run_interactive_chat()             - Start interactive chat")
print("3. demo_conversation()                - Run demo with context")
print("4. quick_chat(['q1', 'q2', 'q3'])     - Chat with question list")
print("5. analyze_conversation(history)      - Analyze conversation")
print("6. save_conversation(history, 'file') - Save to JSON file")
print("\n🚀 Ready to go! Try any of these commands in the next cell.")

## 🚀 Start Here!

Use the cell below to start interacting with Agent 007. Try any of the commands listed above!

In [ ]:
# Uncomment and run any of these commands:

# For a quick question:
# ask_bot("Tell me about artificial intelligence")

# For interactive chat (works best locally):
# run_interactive_chat()

# For a demo conversation:
# demo_conversation()

# Your turn - try something!
print("✨ Agent 007 is ready for your mission!")
print("Choose a command from above and uncomment it, or write your own!")